In [2]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import os

def create_excel(data, path):
    if os.path.exists(path):
        df = pd.read_excel(path)
    else:
        landmark_labels = ['nose', 'left_eye_inner', 'left_eye', 'left_eye_outer', 'right_eye_inner', 'right_eye',
                           'right_eye_outer', 'left_ear', 'right_ear', 'mouth_left', 'mouth_right', 'left_shoulder',
                           'right_shoulder', 'left_elbow', 'right_elbow', 'left_wrist', 'right_wrist', 'left_pinky',
                           'right_pinky', 'left_index', 'right_index', 'left_thumb', 'right_thumb', 'left_hip', 
                           'right_hip', 'left_knee', 'right_knee', 'left_ankle', 'right_ankle', 'left_heel', 
                           'right_heel', 'left_foot_index', 'right_foot_index']
        df = pd.DataFrame(columns=landmark_labels)

    new_row = {}
    for label, x, y, z in data:
        new_row[label] = f"({x}, {y}, {z})"

    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    df.to_excel(path, index=False)

cap = cv2.VideoCapture('Kata_F2_Right.avi')

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7)

mp_drawing = mp.solutions.drawing_utils

polygon_vertices_percent = [(0.1, 0.1), (0.9, 0.1), (0.9, 0.9), (0.1, 0.9)]

def percent_to_pixel(width, height, percentages):
    return [(int(width * x), int(height * y)) for x, y in percentages]

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

polygon_vertices = percent_to_pixel(frame_width, frame_height, polygon_vertices_percent)

def point_inside_polygon(x, y, poly):
    n = len(poly)
    inside = False

    p1x, p1y = poly[0]
    for i in range(1, n + 1):
        p2x, p2y = poly[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xinters:
                        inside = not inside
        p1x, p1y = p2x, p2y

    return inside

landmark_labels = {
    0: 'nose',
    1: 'left_eye_inner',
    2: 'left_eye',
    3: 'left_eye_outer',
    4: 'right_eye_inner',
    5: 'right_eye',
    6: 'right_eye_outer',
    7: 'left_ear',
    8: 'right_ear',
    9: 'mouth_left',
    10: 'mouth_right',
    11: 'left_shoulder',
    12: 'right_shoulder',
    13: 'left_elbow',
    14: 'right_elbow',
    15: 'left_wrist',
    16: 'right_wrist',
    17: 'left_pinky',
    18: 'right_pinky',
    19: 'left_index',
    20: 'right_index',
    21: 'left_thumb',
    22: 'right_thumb',
    23: 'left_hip',
    24: 'right_hip',
    25: 'left_knee',
    26: 'right_knee',
    27: 'left_ankle',
    28: 'right_ankle',
    29: 'left_heel',
    30: 'right_heel',
    31: 'left_foot_index',
    32: 'right_foot_index'
}

# Output directory for depth frames
output_depth_frames_dir = "depth_frames_output"
if not os.path.exists(output_depth_frames_dir):
    os.makedirs(output_depth_frames_dir)

depth_frames = []

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break

    frame_with_overlay = frame.copy()
    RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(RGB)

    if results.pose_landmarks:
        labeled_landmark_coordinates = []

        for landmark_id, landmark in enumerate(results.pose_landmarks.landmark):
            x = int(landmark.x * frame_width)
            y = int(landmark.y * frame_height)

            if point_inside_polygon(x, y, polygon_vertices):
                z = landmark.z * frame_width  # Assuming depth is scaled similarly
                label = landmark_labels.get(landmark_id, f'landmark_{landmark_id}')
                labeled_landmark_coordinates.append((label, x, y, z))

        for label, x, y, z in labeled_landmark_coordinates:
            cv2.circle(frame_with_overlay, (x, y), 5, (255, 0, 0), -1)
            cv2.putText(frame_with_overlay, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1, cv2.LINE_AA)

        if labeled_landmark_coordinates:
            min_x = min(labeled_landmark_coordinates, key=lambda x: x[1])[1]
            max_x = max(labeled_landmark_coordinates, key=lambda x: x[1])[1]
            min_y = min(labeled_landmark_coordinates, key=lambda x: x[2])[2]
            max_y = max(labeled_landmark_coordinates, key=lambda x: x[2])[2]

            cv2.rectangle(frame_with_overlay, (min_x, min_y), (max_x, max_y), (0, 0, 255), 2)

            print(f'Labeled landmark positions: {labeled_landmark_coordinates}')
            create_excel(labeled_landmark_coordinates, "karate_kata_depth2.xlsx")

    cv2.polylines(frame_with_overlay, [np.array(polygon_vertices)], isClosed=True, color=(0, 255, 0), thickness=2)
    cv2.imshow('Karate Kata Analysis', frame_with_overlay)

    # Save depth frame
    depth_frames.append(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Save depth frames to separate files
for i, depth_frame in enumerate(depth_frames):
    depth_frame_path = os.path.join(output_depth_frames_dir, f"depth_frame_{i}.png")
    cv2.imwrite(depth_frame_path, depth_frame)
    print(f"Depth frame {i} saved as {depth_frame_path}")

cap.release()
cv2.destroyAllWindows()



c:\Users\WW\anaconda3\envs\pose_env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\WW\anaconda3\envs\pose_env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\WW\anaconda3\envs\pose_env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\WW\anac

Labeled landmark positions: [('nose', 241, 148, -133.59153747558594), ('left_eye_inner', 243, 144, -125.93226623535156), ('left_eye', 244, 144, -125.942626953125), ('left_eye_outer', 245, 143, -125.95331573486328), ('right_eye_inner', 239, 144, -126.67676544189453), ('right_eye', 238, 144, -126.69044494628906), ('right_eye_outer', 237, 144, -126.67948913574219), ('left_ear', 246, 144, -74.41715240478516), ('right_ear', 234, 145, -78.10440063476562), ('mouth_left', 244, 151, -114.04296112060547), ('mouth_right', 239, 151, -115.0717544555664), ('left_shoulder', 260, 163, -35.325836181640625), ('right_shoulder', 222, 162, -38.93840789794922), ('left_elbow', 285, 163, -20.53237533569336), ('right_elbow', 198, 161, -22.48073959350586), ('left_wrist', 308, 161, -65.20247650146484), ('right_wrist', 173, 161, -69.98434448242188), ('left_pinky', 314, 163, -74.66149139404297), ('right_pinky', 166, 162, -81.56416320800781), ('left_index', 315, 162, -90.69102478027344), ('right_index', 166, 161, -

c:\Users\WW\anaconda3\envs\pose_env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\WW\anaconda3\envs\pose_env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\WW\anaconda3\envs\pose_env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Labeled landmark positions: [('nose', 242, 147, -156.21630859375), ('left_eye_inner', 243, 143, -149.07357788085938), ('left_eye', 244, 143, -149.08135986328125), ('left_eye_outer', 245, 143, -149.107177734375), ('right_eye_inner', 240, 143, -149.94525146484375), ('right_eye', 238, 143, -149.97682189941406), ('right_eye_outer', 237, 143, -149.97201538085938), ('left_ear', 247, 144, -98.56724548339844), ('right_ear', 234, 145, -102.49566650390625), ('mouth_left', 244, 150, -136.82228088378906), ('mouth_right', 240, 151, -137.972412109375), ('left_shoulder', 260, 163, -58.834373474121094), ('right_shoulder', 222, 162, -64.55604553222656), ('left_elbow', 286, 164, -52.122920989990234), ('right_elbow', 196, 161, -60.09463882446289), ('left_wrist', 310, 163, -94.29190826416016), ('right_wrist', 172, 161, -106.19689178466797), ('left_pinky', 316, 164, -103.72084045410156), ('right_pinky', 165, 162, -118.48653411865234), ('left_index', 317, 164, -119.84281158447266), ('right_index', 165, 161,

c:\Users\WW\anaconda3\envs\pose_env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\WW\anaconda3\envs\pose_env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\WW\anaconda3\envs\pose_env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


...

Labeled landmark positions: [('nose', 259, 167, -201.88150024414062), ('left_eye_inner', 261, 161, -191.4375762939453), ('left_eye', 263, 161, -191.47068786621094), ('left_eye_outer', 265, 161, -191.49649047851562), ('right_eye_inner', 255, 161, -192.25357055664062), ('right_eye', 253, 161, -192.30120849609375), ('right_eye_outer', 251, 161, -192.30023193359375), ('left_ear', 268, 163, -122.0909652709961), ('right_ear', 247, 163, -125.30448913574219), ('mouth_left', 262, 173, -175.4473876953125), ('mouth_right', 254, 172, -176.40603637695312), ('left_shoulder', 284, 193, -80.5260009765625), ('right_shoulder', 232, 193, -81.05237579345703), ('left_elbow', 282, 228, -135.01388549804688), ('right_elbow', 233, 228, -135.68190002441406), ('left_wrist', 281, 233, -263.7308349609375), ('right_wrist', 236, 233, -259.429931640625), ('left_pinky', 280, 238, -292.5975341796875), ('right_pinky', 236, 237, -286.2690734863281), ('left_index', 278, 234, -302.406982421875), ('right_index', 237, 232, -

c:\Users\WW\anaconda3\envs\pose_env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Labeled landmark positions: [('nose', 259, 167, -208.1936798095703), ('left_eye_inner', 261, 161, -198.07769775390625), ('left_eye', 263, 161, -198.10540771484375), ('left_eye_outer', 265, 161, -198.12606811523438), ('right_eye_inner', 255, 161, -198.7041778564453), ('right_eye', 253, 161, -198.75433349609375), ('right_eye_outer', 251, 161, -198.7514190673828), ('left_ear', 268, 163, -129.1494598388672), ('right_ear', 247, 163, -130.7378692626953), ('mouth_left', 262, 173, -181.6180877685547), ('mouth_right', 255, 172, -182.18075561523438), ('left_shoulder', 285, 193, -86.90226745605469), ('right_shoulder', 232, 193, -84.73489379882812), ('left_elbow', 283, 229, -142.99478149414062), ('right_elbow', 230, 229, -138.96620178222656), ('left_wrist', 280, 235, -268.1937561035156), ('right_wrist', 236, 235, -262.23651123046875), ('left_pinky', 279, 239, -297.3174133300781), ('right_pinky', 236, 239, -288.8663024902344), ('left_index', 277, 235, -306.14801025390625), ('right_index', 237, 235,

c:\Users\WW\anaconda3\envs\pose_env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Labeled landmark positions: [('nose', 259, 167, -210.0199737548828), ('left_eye_inner', 261, 161, -199.5030517578125), ('left_eye', 263, 161, -199.5259552001953), ('left_eye_outer', 265, 161, -199.53762817382812), ('right_eye_inner', 256, 161, -200.09303283691406), ('right_eye', 253, 161, -200.13917541503906), ('right_eye_outer', 251, 161, -200.1327362060547), ('left_ear', 268, 163, -128.7171173095703), ('right_ear', 247, 163, -130.1546630859375), ('mouth_left', 262, 173, -182.82720947265625), ('mouth_right', 255, 173, -183.33694458007812), ('left_shoulder', 285, 193, -87.96932983398438), ('right_shoulder', 231, 193, -86.03455352783203), ('left_elbow', 285, 227, -144.56451416015625), ('right_elbow', 227, 227, -142.08755493164062), ('left_wrist', 281, 234, -292.7531433105469), ('right_wrist', 235, 236, -286.2363586425781), ('left_pinky', 281, 240, -325.7701721191406), ('right_pinky', 235, 240, -316.7904052734375), ('left_index', 279, 237, -332.4170227050781), ('right_index', 237, 237, -